In [15]:
import pandas as pd
import numpy as np
import datetime

### Reading the Data

In [17]:
train_df = pd.read_csv("git\sign_mnist_train.csv")
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


AttributeError: 'Series' object has no attribute 'dist'

In [3]:
test_df = pd.read_csv("git\sign_mnist_test.csv")
test_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53


In [4]:
categories = train_df["label"].unique()
categories.sort()
categories.shape

(24,)

### Preprocessing the Data

In [5]:
from sklearn.preprocessing import MinMaxScaler
train_df = train_df.sample(frac=1)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df.iloc[:,1:])
y_train = train_df.iloc[:,0]

In [6]:
test_df = test_df.sample(frac=1)
x_test =  scaler.fit_transform(test_df.iloc[:,1:])
y_test = test_df.iloc[:,0]

In [7]:
del train_df
del test_df

### Training the Model

In [8]:
TRAIN_LEN = x_train.shape[0]

from sklearn import svm
s = datetime.datetime.now()
clf = svm.SVC(verbose = True)
clf.fit(x_train[:TRAIN_LEN],y_train.iloc[:TRAIN_LEN])
datetime.datetime.now() - s 

[LibSVM]

datetime.timedelta(seconds=112, microseconds=95043)

### Checking the Accuracy

In [10]:
TEST_LEN = x_test.shape[0]
pred = clf.predict(x_test[:TEST_LEN])
"Acc. in " + str(TEST_LEN) + " test samples: "  + "%" + str(100 * np.array([pred == y_test.iloc[:TEST_LEN]]).sum()/TEST_LEN)

'Acc. in 7172 test samples: %83.81204684885667'

### Testing a Single Sample With Visualization

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.pyplot import imshow

In [ ]:
instance = 80 
a = x_test[instance].reshape(1,-1)
a_image = a.reshape((28,28))

pred_a = clf.predict(a)
im = Image.fromarray((a_image.astype(np.uint8)))

imshow(np.asarray(a_image))
print("--------------")
print("The model predicted this instance as label", pred[instance])
print("--------------")
print("Where its label is", y_test.iloc[instance])

In [11]:
categories

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)